### 固有状態

系の時間発展を記述する演算子$H$ (行列)をハミルトニアンといい、時間発展に対応する偏微分方程式をSchroedinger方程式という。
$$
 i \frac{\partial }{\partial t} |\psi(t) \rangle = H |\psi(t)\rangle.
$$
ここで、この方程式は$H$の固有ベクトル、固有エネルギーがわかれば完全に解けるので、固有状態を求めるタスクは重要。

特に、$H$が実数成分のみで与えられている場合には、$H$は直交行列のみを用いて対角化できるので、自然と固有ベクトルも実数係数のみで書ける。

第一原理的に考えると、量子力学のミクロな世界は時間反転対称性があるはずで、そのような場合にはハミルトニアンは実数成分のみで与えられることが知られているらしい。たとえば[この文献](https://topocondmat.org/w1_topointro/0d.html#time-reversal-symmetry)は参考になりそう。

実際には、物理では有効模型を抽出して物事を調べることが多く、必ずしも時間反転対称性が守られたハミルトニアンを考えるわけではないが、以下のような例では時間反転対称性が保たれ、実成分のみで書ける。
- ハイゼンベルク模型
- 横磁場イジング模型
- フェルミ-ハバード模型

In [1]:
from openfermion import QubitOperator, FermionOperator, get_sparse_operator
import numpy as np
import scipy


def _heisen(i, j):
    return (
        QubitOperator(f"X{i} X{j}")
        + QubitOperator(f"Y{i} Y{j}")
        + QubitOperator(f"Z{i} Z{j}")
    )


def HeisenbergHamiltonian(
    edges,
):
    """
    ハイゼンベルク模型
        H = \sum_{<i, j>} X_i X_j + Y_i Y_j + Z_i Z_j
    attributes:
        edges : 相互作用のあるエッジ
    """
    ham = QubitOperator()
    for e in edges:
        ham += _heisen(*e)
    return ham


def TransversalFieldIsingHamiltonian(edges, hx):
    """
    横磁場イジング模型
       H = h_x \sum_i X_i + \sum_{<i, j>} Z_i Z_j
    attributes:
        edges : 相互作用のあるエッジ
        hx       : X方向の磁場の大きさ
    """
    ham = QubitOperator()
    n_qubit = max(max(e) for e in edges) + 1
    for i in range(n_qubit):
        ham += hx * QubitOperator(f"X{i}")
    for e in edges:
        ham += QubitOperator(f"Z{e[0]} Z{e[1]}")
    return ham


def FermiHubbardHamiltonian(edges_hop, edges_int, interaction, potential):
    """
    フェルミハバード模型 (最近接ホッピングのみ、オンサイトポテンシャルのみ)
       H = H_hop + H_{pot} + H_int
         H_hop = \sum_{<i, j> : edges_hop} c_i^\dag c_j + c_j^\dag c_j
         H_pot = (potential) * \sum_i c_i^\dag c_i
         H_int = (interaction) * \sum_{<i, j> : edges_int} c_i^\dag c_i c_j^\dag c_j
    attributes:
        edges_hop : ホッピングを許すエッジ
        edges_int : 相互作用を許すエッジ
        interawction : 相互作用の強さ
        potential : 各サイトの化学ポテンシャル
    """
    ham = FermionOperator()
    n_mode = max(max(e) for e in edges) + 1

    for i in range(n_qubit):
        ham += potential * FermionOperator(f"{i}^ {i}")
    for e in edges_int:
        i, j = e
        ham += interaction * FermionOperator(f"{i}^ {i} {j}^ {j}")

    for e in edges_hop:
        i, j = e
        ham += FermionOperator(f"{i}^ {j}") + FermionOperator(f"{j}^ {i}")

    return ham

In [9]:
# Heisenberg

n_qubit = 4
edges = [(0, 1), (1, 2), (2, 3)]
hamiltonian = HeisenbergHamiltonian(edges)

ham_sp = get_sparse_operator(hamiltonian)
ham_mat = ham_sp.toarray()

vals, vecs_tmp = np.linalg.eigh(ham_mat)
eigenvectors = vecs_tmp.T
print(
    "is eigenvectors for Heisenberg Hamiltonian real? ",
    np.allclose(eigenvectors, np.real(eigenvectors), atol=1e-10),
)

(16, 16)
is eigenvectors for Heisenberg Hamiltonian real?  True


In [5]:
# Transverse-field Ising (TFI)

n_qubit = 4
hx = 0.3
edges = [(0, 1), (1, 2), (2, 3)]
hamiltonian = TransversalFieldIsingHamiltonian(edges, hx)

ham_sp = get_sparse_operator(hamiltonian)
ham_mat = ham_sp.toarray()

vals, vecs_tmp = np.linalg.eigh(ham_mat)
eigenvectors = vecs_tmp.T
print(
    "is eigenvectors for TFI Hamiltonian real? ",
    np.allclose(eigenvectors, np.real(eigenvectors), atol=1e-10),
)

is eigenvectors for TFI Hamiltonian real?  True


In [19]:
# Fermi-Hubbard 模型

n_qubit = 4
edges_hop = [(0, 2), (1, 3)]
edges_int = [(0, 1), (2, 3)]
potential = 2.0
interaction = 1.0
hamiltonian = FermiHubbardHamiltonian(edges_hop, edges_int, potential, interaction)

ham_sp = get_sparse_operator(hamiltonian)
ham_mat = ham_sp.toarray()

vals, vecs_tmp = np.linalg.eigh(ham_mat)
eigenvectors = vecs_tmp.T
print(
    "is eigenvectors for Fermi-Hubbard Hamiltonian real? ",
    np.allclose(eigenvectors, np.real(eigenvectors), atol=1e-10),
)

is eigenvectors for Fermi-Hubbard Hamiltonian real?  True


### Gibbs 状態
Hamiltonianが$H = \sum_i E_i |\psi_i \rangle \langle \psi_i |$で記述されるような系のGibbs状態 (熱平衡状態)は、逆温度$\beta = 1/k_B T$($T$:温度, $k_B$:ボルツマン定数)に対して
$$
    \rho := e^{- \beta H}/ Z = \sum_i p_i |\psi_i\rangle \langle \psi_i |
$$
で与えられる。ここで、$Z = {\rm Tr}[e^{-\beta H}] = \sum_i e^{- \beta E_i}$はボルツマン因子、$p_i = e^{- \beta E_i}/Z$は規格化されたボルツマン重みである。

上に述べたように、固有ベクトルが実数係数のみで書けている場合は、そのprobabilistic mixtureであるGibbs状態も、同様に実数成分をもつ密度行列になる。

In [6]:
beta = 1.0  # inverse temperature
gibbs_state = scipy.linalg.expm(-beta * ham_mat)
gibbs_state /= np.trace(gibbs_state)
print(
    "is Gibbs state real? ", np.allclose(gibbs_state, np.real(gibbs_state), atol=1e-10)
)

(16, 16)
is Gibbs state real?  True


In [7]:
# check the construction of Gibbs state.

# calculation 1 : <O> = Tr[\rho O]
observable = QubitOperator("X0 X1")
obs_mat = get_sparse_operator(observable, n_qubits=n_qubit).toarray()
gibbs_expectation_value1 = np.trace(gibbs_state @ obs_mat)

# calculation 2 : <O> = \sum_i p_i < psi_i | O | psi_i> =
probability = np.exp(-beta * vals)
probability /= sum(probability)
expectation_values = [_vec.conj() @ obs_mat @ _vec for _vec in eigenvectors]
gibbs_expectation_value2 = sum(
    [probability[i] * expectation_values[i] for i in range(len(eigenvectors))]
)
print(
    "consistency check in calculating expectation values: ",
    np.isclose(gibbs_expectation_value1, gibbs_expectation_value2),
)

consistency check in calculating expectation values:  True
